In [24]:
#FIRST of all, install SSMS (SQL Server MicroSoft)
#Second of all, open the SQLQuery1.sql file in SqlServer
#Thrid of all, highlight ONLY the first command (create database Brokerage) then click Execute
#Fourth of all, highlight the rest of the file lines then click Execute.
#by now you should have the database and schema set and done

#now you will need to populate it with data

#Fifth of all, install pyodbc (nice library btw)

#you will then want to connect your database to the ODBC:
#   1) open SQL Server
#   2) right click on the very first element in object explorer (which is your device obvs)
#   3) choose connect
#   4) copy the server name
#   5) paste it in the server string below


#Sixth of all, just run everything below and enjoy :)

#OPTIONAL : you don't need to know how to generate the report file, it is so ez though
#   just open SSMS
#   Select the database "Brokerage"
#   Right click and select Reports then Standard Reports
#   Select any of the reports than seems interesting

In [1]:
import pandas as pd
import pyodbc
import numpy as np
import random
from tqdm import tqdm


In [2]:
#Helper Functions



alphabet = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

#to return a random Department Name
def GenerateRandName(digits):
    Random_dept_name = ''
    max_len = random.choice(range(9,40))
    while len(Random_dept_name) < max_len:
        if(digits):
            Random_dept_name = Random_dept_name+(random.choice(alphabet+'123456789'))
        else:
            Random_dept_name = Random_dept_name+(random.choice(alphabet))
    return Random_dept_name

def GenerateRandomDate():
    randDay = random.choice(range(1,29))
    randMonth = random.choice(range(1,12))
    randYear = random.choice(range(2000,2023))
    return pd.to_datetime(str(randDay) + "/"+str(randMonth)+"/"+str(randYear), dayfirst=True)


In [3]:
for driver in pyodbc.drivers():
    print(driver)

SQL Server
SQL Server Native Client 11.0
SQL Server Native Client RDA 11.0
ODBC Driver 17 for SQL Server
Microsoft Access Driver (*.mdb, *.accdb)
Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)
Microsoft Access Text Driver (*.txt, *.csv)


In [4]:
df = pd.read_csv('Employee.csv')
#Columns in the CSV : first_name Minit last_name Address gender BDate Salary
#attributes in our database: Fname Minit Lname Bdate Address_ Sex Salary
#print(df['first_name'])

#for i in range(1000):
#    df.loc[i,'Minit'] = random.choice(alphabet)
#df.to_csv('MOCK_DATA.csv')

#print(df.iloc[:,:].values.tolist())


In [4]:
#QUERIES TO FILL "Employee"


server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm_10k'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()

cursor.execute('delete from Employee')
connection.commit()

df = pd.read_csv('Employee.csv')
print(df.head())

insert_to_employee = 'INSERT INTO Employee (Fname, Minit, Lname, Address_, Bdate, Sex, Salary) VALUES (?,?,?, ?, ?, ?, ?)'
select_all_from_employee = "select * from Employee"

for i in tqdm(range(30000), desc="inserting data into Table Employee: "):
    lst = df.iloc[i,1:].values.tolist()
    cursor.execute(insert_to_employee, lst)

connection.commit()

get_Employees_count = "select count (*) from Employee"

cursor = connection.cursor()
count = cursor.execute(get_Employees_count)
print(list(count))


   Unnamed: 0 first_name Minit  last_name  \
0           0       Mary     a       Anna   
1           1       Anna     b       Emma   
2           2       Emma     c  Elizabeth   
3           3  Elizabeth     d     Minnie   
4           4     Minnie     e   Margaret   

                                Address       BDate  Sex   Salary  
0           7353 HIGHLAND RD, STE B 282  2000-01-01  0.0      0.0  
1               9007 HIGHLAND RD, STE 9  2001-02-02  1.0   3000.0  
2   5830 S SHERWOOD FOREST BLVD, STE A6  2002-03-03  2.0   6000.0  
3  4520 S SHERWOOD FOREST BLVD, STE 103  2003-04-04  0.0   9000.0  
4                 8334 O'HARA CT, STE D  2004-05-05  1.0  12000.0  


inserting data into Table Employee: 100%|██████████| 30000/30000 [00:13<00:00, 2289.35it/s]

[(30000, )]


In [18]:
''' Department Attributes
Dname nvarchar(50),
Dnumber int identity (1,1) Primary Key NOT NULL,
Mgr_SSN int foreign key references Employee(SSN),
Mgr_Start_Date DATE
'''
Dept_df = pd.DataFrame()
Dept_df['Dname'] = GenerateRandName(True)
Dept_df['Mgr_SSN'] = random.choice(range(1,30))
Dept_df['Mgr_Start_Date'] = pd.to_datetime('11/11/2000', dayfirst=True)

for i in tqdm(range(0,10000), desc="making Department.csv: "):
    Dept_df.loc[i, 'Dname'] = GenerateRandName(True)
    Dept_df.loc[i, 'Mgr_SSN'] = i+13000
    Dept_df.loc[i, 'Mgr_Start_Date'] = GenerateRandomDate()
    


while(len(Dept_df) < 120000):
    Dept_df = pd.concat((Dept_df, Dept_df),axis=0)
Dept_df.to_csv('Department_10k.csv')

making Department.csv: 100%|██████████| 10000/10000 [00:14<00:00, 697.93it/s]


In [13]:
connection.commit()
cursor.execute('delete from Department')
connection.commit()

data = cursor.execute('select * from Employee where SSN = 30')
print(list(data))

[]


In [5]:
server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm_10k'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()

cursor.execute('delete from Department')
connection.commit()

Dept_df = pd.read_csv('Department_10k.csv')
print(Dept_df.head())

insert_to_Department= "INSERT INTO Department (Dname, Mgr_SSN, Mgr_Start_Date) VALUES(?,?, ?)"
select_all_from_Department = "select * from Department"
get_Department_count = "select count (*) from Department"

for i in tqdm(range (10000), desc="inserting data into Department: "):
    lst = Dept_df.iloc[i,1:].values.tolist()
    cursor.execute(insert_to_Department, lst)
    

connection.commit()

count = cursor.execute(get_Department_count)
print(list(count))

   Unnamed: 0                                 Dname  Mgr_SSN Mgr_Start_Date
0           0  rlr6ePaJfWTbqUEl98CgtzKoYy4TB2LYIdUR  13000.0     2022-06-20
1           1        t6DWwYfZlkenX6lOoSFcmEfcTqCkw9  13001.0     2017-09-08
2           2    zdonJ7Rekg4BiqVOIHnwJHRCGx5P5oVmIQ  13002.0     2011-08-24
3           3                     oANIFNupletKFy8XG  13003.0     2018-08-05
4           4                          VcAZFY7EhWIW  13004.0     2001-09-11


inserting data into Department: 100%|██████████| 10000/10000 [00:02<00:00, 3772.36it/s]

[(10000, )]


In [3]:
''' DEPT LOCATION

create table Dept_Locations
(
Dnumber INT foreign key references Department(Dnumber),
Dlocation nvarchar(50) NOT NULL,
CONSTRAINT Department_KEY_COMPOSITE PRIMARY KEY (Dnumber, Dlocation)
);
'''

DeptLoc_df = pd.DataFrame()
Address_df = pd.read_csv('Address.csv')

temp_df = DeptLoc_df
for i in tqdm(range(900000), desc="Making Dept_Locations.csv: "):
    temp_df.loc[i,'Dnumber'] = i+1
    temp_df.loc[i,'Dlocation'] = Address_df.loc[random.choice(range(1,400000)), 'FULL ADDRESS']
    if(i%20000 == 0):
        DeptLoc_df = pd.concat((DeptLoc_df, temp_df), axis=0)
        temp_df = pd.DataFrame()
        DeptLoc_df.to_csv('Dept_Locations.csv')



DeptLoc_df = pd.concat((DeptLoc_df, temp_df), axis=0)
DeptLoc_df.to_csv('Dept_Locations.csv')

Making Dept_Locations.csv: 100%|██████████| 900000/900000 [17:09<00:00, 873.97it/s]  


In [17]:
connection.commit()
cursor.execute('Delete from Dept_Locations')
connection.commit()



In [7]:

server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm_10k'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()

DeptLoc_df = pd.read_csv('Dept_Locations.csv')
print(DeptLoc_df.head())

insert_to_DeptLocation = "INSERT INTO Dept_Locations (Dlocation) VALUES(?)"
select_all_from_DeptLocations = "select * from Dept_Locations"
get_Dept_location_count = "select count (*) from Dept_Locations"



for i in tqdm(range(10),desc="inserting into Dept_Locations: "):
    lst = DeptLoc_df.iloc[i*1000:(i+1)*1000,2].values.tolist()
    for s in lst:
        try:
            cursor.execute(insert_to_DeptLocation, s)
        except:
            pass
   


connection.commit()

count = cursor.execute(get_Dept_location_count)
print(list(count))

   Unnamed: 0  Dnumber                  Dlocation
0           1      1.0          10082 E POMONA DR
1           2      2.0          16933 TENEIYA AVE
2           3      3.0          13630 LANDMARK DR
3           4      4.0          11733 SUN BELT CT
4           5      5.0  17100-17400 BLACKWATER RD


inserting into Dept_Locations: 100%|██████████| 10/10 [00:00<00:00, 10.95it/s]

[(10000, )]


In [16]:


''' Intern TABLE ATTRIBUTES
(
SSN int identity (3000000,1) primary key Not NULL,
Fname nvarchar(50),
Minit varchar,
Lname nvarchar(50),
Bdate DATE,
Address_ nvarchar(50),
Sex tinyint check(Sex between 0 and 2),
Salary int,
Major nvarchar(50),
PNO INT foreign key references Project(Pnumber),
Super_SSN int foreign key references Employee(SSN)
);
'''

names_df = pd.read_csv('Names.csv')
Address_df = pd.read_csv('Address.csv')
jobs_df = pd.read_csv('Jobs.csv')

#new empty dataframe
df_intern = pd.DataFrame()
#df_intern['SSN']=         auto incremented
df_intern['Fname']= ['Ayman', 'Mohamed']
df_intern['Minit']= ['M', 'R']
df_intern['Lname']= ['Reda', 'Abdo']
df_intern['Bdate']= [pd.to_datetime("20/12/2004", dayfirst=True), pd.to_datetime("11/11/2000", dayfirst=True)]
df_intern['Address_']= [Address_df.loc[1, 'FULL ADDRESS'], Address_df.loc[3, 'FULL ADDRESS']]
df_intern['Sex']= [1,1]
df_intern['Salary']= [2000,3000]
df_intern['Major']= ["Computer", "Engineering"]
df_intern['Super_SSN']= [1, 2]


temp_df = df_intern
for i in tqdm(range(15000),desc="making Intern.csv: "):
    temp_df.loc[i, 'Fname'] = names_df.loc[i%1000000, 'Name']
    temp_df.loc[i, 'Minit'] = random.choice(alphabet)
    temp_df.loc[i, 'Lname'] = names_df.loc[i%1000000, 'Name']
    temp_df.loc[i, 'Address_'] = Address_df.loc[i%1000000, 'FULL ADDRESS']
    
    randSex = random.choice(range(0,3))
    randSalary = random.choice(range(1, 100000))
    
    temp_df.loc[i, 'Bdate'] = GenerateRandomDate()
    temp_df.loc[i, 'Sex'] = randSex
    temp_df.loc[i, 'Salary'] = randSalary
    temp_df.loc[i, 'Major'] = jobs_df.loc[i%5000, 'top related titles 19']
    temp_df.loc[i, 'Super_SSN'] = random.choice(range(1,10000))                                  #the first 500000 employees are intern supervisors
    

df_intern = pd.concat((df_intern, temp_df), axis=0)
df_intern.to_csv('Intern_10k.csv')

making Intern.csv: 100%|██████████| 15000/15000 [00:31<00:00, 474.19it/s]


In [20]:

server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm_10k'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()

df_intern = pd.read_csv('Intern_10k.csv')
print(df_intern.head())

insert_to_Intern = "INSERT INTO Intern (Fname, Minit, Lname, Bdate, Address_, Sex, Salary, Major, Super_SSN) VALUES(?,?, ?, ?, ?, ?, ?, ?, ?)"
select_all_from_Intern = "select * from Intern"
get_Intern_count = "select count (*) from Intern"
get_Employees_count = "select count (*) from Employee"

for i in tqdm(range(10000),desc="inserting into Intern: "):
    lst = df_intern.iloc[i,1:].values.tolist()
    try:
        cursor.execute(insert_to_Intern, lst)
    except:
        pass
    

connection.commit()

count = cursor.execute(get_Employees_count)
print(list(count))

   Unnamed: 0      Fname Minit      Lname       Bdate  \
0           0       Mary     b       Mary  2000-04-19   
1           1       Anna     Y       Anna  2019-11-18   
2           2       Emma     p       Emma  2007-07-24   
3           3  Elizabeth     N  Elizabeth  2004-06-03   
4           4     Minnie     e     Minnie  2011-09-25   

                               Address_  Sex   Salary                  Major  \
0           7353 HIGHLAND RD, STE B 282  0.0  36081.0        general manager   
1               9007 HIGHLAND RD, STE 9  1.0  10166.0          sales manager   
2   5830 S SHERWOOD FOREST BLVD, STE A6  1.0  43454.0              principal   
3  4520 S SHERWOOD FOREST BLVD, STE 103  2.0  67965.0                  owner   
4                 8334 O'HARA CT, STE D  1.0  23118.0   chief executive offi   

   Super_SSN  
0     8074.0  
1     4816.0  
2     3209.0  
3     7263.0  
4     7984.0  


inserting into Intern: 100%|██████████| 10000/10000 [00:17<00:00, 567.67it/s]

[(30000, )]


In [21]:

server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm_10k'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()

project_df = pd.read_csv('Project.csv')
print(project_df.head())

insert_to_Project = 'INSERT INTO Project (Pname, Plocation, DNO) VALUES (?,?, ?)'
select_all_from_Project = "select * from Project"

for i in tqdm(range(10), desc="inserting data into Table Project: "):
    lst = project_df.iloc[(i)*1000: (i+1)*1000,1:].values.tolist()
    cursor.executemany(insert_to_Project, lst)

connection.commit()

get_Project_count = "select count (*) from Project"

cursor = connection.cursor()
count = cursor.execute(get_Project_count)
print(list(count))

   Unnamed: 0                                           Pname  \
0           0                              advanced databases   
1           1                                              9F   
2           2                           LEZ3SwJIykqigwhJ1bPw7   
3           3             Vyp3AQZxOF4zQNki7hAU5m5B7X4gY1o3cy6   
4           4  ffI4FhNEyXiK9ri5QToVRIBkxg5pUCgOV1m7BB868r2nm8   

                              Plocation  DNO  
0           7353 HIGHLAND RD, STE B 282  1.0  
1               9007 HIGHLAND RD, STE 9  2.0  
2   5830 S SHERWOOD FOREST BLVD, STE A6  3.0  
3  4520 S SHERWOOD FOREST BLVD, STE 103  4.0  
4                 8334 O'HARA CT, STE D  5.0  


inserting data into Table Project: 100%|██████████| 10/10 [00:03<00:00,  3.07it/s]

[(10000, )]


In [ ]:
'''

create table Works_On
(
ESSN INT foreign key references Employee(SSN),
PNO INT foreign key references Project(Pnumber),
Hours INT,
CONSTRAINT WorksOn_KEY_COMPOSITE PRIMARY KEY (ESSN, PNO)
);

'''

#new empty dataframe
df_workson = pd.Dataframe()

df_workson[0,'ESSN']= 1
df_workson[0,'PNO']= 1
df_workson[0,'Hours']= 19

PNO_counter = 1
tempdf = df_workson
for i in tqdm(range(2900000), desc="making Works_On.csv: "):
    tempdf.loc[i, 'ESSN'] = i+1
    tempdf.loc[i, 'PNO'] = PNO_counter
    temp_df.loc[i, 'Hours'] = random.randint(15, 50)
    if(i%17000 == 0):
        PNO_counter = PNO_counter + 1
        df_workson = pd.concat((df_workson, temp_df),axis=0)
        temp_df = pd.DataFrame()
        df_workson.to_csv('Works_On.csv')

df_workson = pd.concat((df_workson, temp_df),axis=0)
df_workson.to_csv('Works_On.csv')

In [24]:
server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm_10k'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()

df_workson = pd.read_csv('Works_On.csv')
print(df_workson.head())

insert_to_Works_On = "INSERT INTO Works_On (ESSN, PNO, Hours) VALUES(?,?,?)"
select_all_from_Works_On = "select * from Works_On"
get_Works_On_count = "select count (*) from Works_On"


for i in tqdm(range(10), desc="inserting into Works_On: "):
    lst = df_workson.iloc[i*1000:(i+1)*1000,1:].values.tolist()
    cursor.executemany(insert_to_Works_On, lst)

connection.commit()

count = cursor.execute(get_Works_On_count)
print(list(count))

   Unnamed: 0  ESSN  PNO  Hours
0           1   1.0  1.0   46.0
1           2   2.0  2.0   44.0
2           3   3.0  2.0   45.0
3           4   4.0  2.0   24.0
4           5   5.0  2.0   20.0


inserting into Works_On: 100%|██████████| 10/10 [00:00<00:00, 12.30it/s]

[(2026, )]


In [ ]:
'''

create table Dependent_
(
ESSN INT foreign key references Employee(SSN),
Dependent_Name nvarchar(50) NOT NULL,
Sex tinyint check(Sex between 0 and 2),
Bdate DATE,
Relationship nvarchar(50),
CONSTRAINT Dependent_KEY_COMPOSITE PRIMARY KEY (ESSN, Dependent_Name)
);
'''


Relationship_df = pd.read_csv('Relationship.csv')

#new empty dataframe
df_Dependent = pd.DataFrame()

df_Dependent[0,'ESSN']= 1
df_Dependent[0,'Dependent_Name']= "Akram"
df_Dependent[0,'Sex']= 1
df_Dependent[0,'Bdate']= pd.to_datetime('11/11/2000', dayfirst=True)
df_Dependent[0,'Relationship']= "Father"




temp_df = df_Dependent
for i in tqdm(range(2900000), desc="making Dependent.csv : "):
    temp_df.loc[i, 'ESSN'] = i+1
    temp_df.loc[i, 'Dependent_Name'] = GenerateRandName(False)
    temp_df.loc[i, 'Sex'] = random.randint(0, 2)
    temp_df.loc[i, 'Bdate'] = GenerateRandomDate()
    temp_df.loc[i, 'Relationship'] = Relationship_df.loc[random.randint(0,970),'Relationship']
    if(i%17000 == 0):
        df_Dependent = pd.concat((df_Dependent, temp_df), axis=0)
        temp_df = pd.DataFrame()
        df_Dependent.to_csv('Dependent.csv')


df_Dependent = pd.concat((df_Dependent, temp_df), axis=0)
df_Dependent.to_csv('Dependent.csv')

In [26]:
connection.commit()
cursor.execute("delete from Dependent_")
connection.commit()

In [10]:
df_Dependent = pd.read_csv('Dependent.csv')
df_Dependent.head()
df_Dependent = df_Dependent.iloc[:, 2:]
df_Dependent.to_csv('Dependent.csv')
df_Dependent = pd.read_csv('Dependent.csv')

In [27]:
server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm_10k'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()

df_Dependent= pd.read_csv('Dependent.csv')
print(df_Dependent.head())

insert_to_Dependent_ = "INSERT INTO Dependent_ (ESSN, Dependent_Name, Sex, Bdate, Relationship) VALUES(?,?,?,?,?)"
select_all_from_Dependent_ = "select * from Dependent_"
get_Dependent__count = "select count (*) from Dependent_"


for i in tqdm(range(10), desc="inserting into Dependent_ : "):
    lst = df_Dependent.iloc[i*1000:(i+1)*1000,2:].values.tolist()
    cursor.executemany(insert_to_Dependent_, lst)

connection.commit()

count = cursor.execute(get_Dependent__count)
print(list(count))

   Unnamed: 0.1  Unnamed: 0  ESSN                      Dependent_Name  Sex  \
0             0           0   1.0        paZKVTQLByxdXhOCpyxXxLUpiqqj  1.0   
1             1           1   2.0       FMQDdqvlxinFUqMVSuGIbqbkLorMc  1.0   
2             2           2   3.0            CjuVAbakpmWAYMFsHoCqobKc  2.0   
3             3           3   4.0  MmGysbOQtdhSRszpXLMzszspphVNUHeuzT  0.0   
4             4           4   5.0                VVXLuargiWnLGulRAkVD  2.0   

        Bdate Relationship  
0  2019-09-06       Lotlux  
1  2001-07-06  Mat Lam Tam  
2  2005-08-17     Wrapsafe  
3  2001-06-24     Treeflex  
4  2017-09-12       Vagram  


inserting into Dependent_ : 100%|██████████| 10/10 [00:01<00:00,  7.43it/s]

[(10000, )]


In [30]:
server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm_10k'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()

Contract_df = pd.read_csv('Contracts.csv')
print(Contract_df.head())

insert_to_Contracts= "INSERT INTO Contracts (Date_created, ProjectID, Product, Price, Date_of_Contract, Emp_Name, Emp_SSN, Contractor_Name, Cont_SSN, Delivery_Location,Name_Company) VALUES(?,?,?, ?,?,?,?,?,?,?,?)"
select_all_from_Contracts = "select * from Contracts"
get_Contracts_count = "select count (*) from Contracts"


for i in tqdm(range(9), desc="inserting data into table Contracts"):
    lst = Contract_df.iloc[i*1000:(i+1)*1000,2:].values.tolist()
    cursor.executemany(insert_to_Contracts, lst)


connection.commit()

count = cursor.execute(get_Contracts_count)
print(list(count))

   Unnamed: 0  (0, 'Name_Company') Date_created  ProjectID  Product  \
0           0                  NaN   2013-05-21        2.0    63999   
1           1                  NaN   2011-10-03        3.0    37891   
2           2                  NaN   2010-02-24        4.0    88587   
3           3                  NaN   2015-08-24        5.0   121650   
4           4                  NaN   2019-02-16        6.0   162283   

       Price Date_of_Contract   Emp_Name  Emp_SSN Contractor_Name   Cont_SSN  \
0  1063907.0       2000-10-21       Anna      1.0            Mary  4000001.0   
1  1381986.0       2000-10-21       Emma      2.0            Anna  4000002.0   
2   339553.0       2000-10-21  Elizabeth      3.0            Emma  4000003.0   
3  1849411.0       2000-10-21     Minnie      4.0       Elizabeth  4000004.0   
4   755588.0       2000-10-21   Margaret      5.0          Minnie  4000005.0   

                      Delivery_Location                           Name_Company  
0          

inserting data into table Contracts: 100%|██████████| 9/9 [00:10<00:00,  1.12s/it]

[(9000, )]


In [31]:
server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm_10k'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()



data = cursor.execute('select count(*) from Employee')
print("size of Employee : " + str(list(data)))

data = cursor.execute('select count(*) from Department')
print("size of Department : " + str(list(data)))


data = cursor.execute('select count(*) from Project')
print("size of Project : " + str(list(data)))

data = cursor.execute('select count(*) from Intern')
print("size of Intern : " + str(list(data)))

data = cursor.execute('select count(*) from Contracts')
print("size of Contracts : " + str(list(data)))

data = cursor.execute('select count(*) from Works_On')
print("size of Works_On : " + str(list(data)))

data = cursor.execute('select count(*) from Dept_Locations')
print("size of Dept_Locations : " + str(list(data)))

size of Employee : [(30000, )]
size of Department : [(10000, )]
size of Project : [(10000, )]
size of Intern : [(2026, )]
size of Contracts : [(9000, )]
size of Works_On : [(10000, )]
size of Dept_Locations : [(10000, )]
